<a href="https://colab.research.google.com/github/nguyenanhtienabcd/M01_PROJECT_RAG_LLM/blob/feature%2FRAG-LLM/M01_Project_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Install libraries

In [1]:
! pip install -q transformers
! pip install -q bitsandbytes
! pip install -q accelerate
! pip install -q langchain
! pip install -q langchainhub
! pip install -q pypdf
! pip install -q numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 4.7 MB/s eta 0:00:00


In [2]:
! pip install -q langchain-chroma
! pip install -q langchain-community
! pip install -q langchain_huggingface
! pip install -q python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8

## 2. import libraries

In [3]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.chains import ConversationalRetrievalChain
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

### Download Yolov10-tutortial

In [ ]:
!gdown '1lWuq0COKnU9mCfMvTEq54DBLgAh3yYDx'

Downloading...
From: https://drive.google.com/uc?id=1lWuq0COKnU9mCfMvTEq54DBLgAh3yYDx
To: /content/YOLOv10_Tutorials.pdf
100% 16.6M/16.6M [00:00<00:00, 41.2MB/s]


## 3. Read file pdf

In [4]:
Loader = PyPDFLoader
FILE_PATH = '/content/Uero_2024_Phap_Bi.pdf'
loader = Loader(FILE_PATH)
documents = loader.load()

In [5]:
# split documents by splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size =1000 ,
                                               chunk_overlap =100)
docs = text_splitter . split_documents ( documents )
print("Number of sub - documents: ", len (docs))
print(docs [0])

Number of sub - documents:  10
page_content='Lần thứ hai tại EURO 2024 đội tuyển  Pháp  giành chiến thắng nhờ bàn phản lưới \nnhà của đối phương, lần này là tuyển Bỉ tại vòng 1/8 EURO 2024 vào rạng sáng \n1.7. \nHẾT GIỜ!!! Tuyển Pháp giành chiến thắng 1 -0 trước Bỉ, do pha phản lưới nhà của \nVertonghen. Tuyển Bỉ đã có một trận đấu hay về  phòng ngự nhưng tấn công quá tẻ \nnhạt. Pháp sẽ gặp Bồ Đào Nha hoặc Slovenia ở tứ kết.  \nHiệp 2 có 3 phút bù giờ.  \nBàn thắng được tính là Jan Vertonghen phả n lưới nhà. Ảnh: UEFA  \nPhút 85:  VÀO!!! Muani thoát xuống và dứt điểm, bóng đập chân Vertonghen đổi hướng \nvà đánh bại thủ thành Casteels, mở tỉ số cho Pháp.' metadata={'source': '/content/Uero_2024_Phap_Bi.pdf', 'page': 0}


## 4. vectorisation: Convert text to vector

In [6]:
embedding = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# create a vector database
vector_db = Chroma.from_documents(documents =docs ,embedding = embedding)
retriever = vector_db.as_retriever()

In [9]:
# queue any short text here
result = retriever.invoke(" Uero 2024, Pháp và Bỉ?")
print(" Number of relevant documents : ", len ( result ))

 Number of relevant documents :  4


## 5. Initialise LLM model

In [10]:
# quantization
nf4_config = BitsAndBytesConfig(load_in_4bit =True ,
                                bnb_4bit_quant_type ="nf4",
                                bnb_4bit_use_double_quant =True ,
                                bnb_4bit_compute_dtype = torch . bfloat16)

In [11]:
 # Initialise model and token
MODEL_NAME = "lmsys/vicuna-7b-v1.5"
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,
                                            quantization_config = nf4_config,
                                            low_cpu_mem_usage = True)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

## 6. Intergrate tokenizer and model into a pipe line

In [12]:
model_pipeline = pipeline('text-generation',
                          model =model,
                          tokenizer = tokenizer,
                          max_new_tokens =512,
                          pad_token_id = tokenizer.eos_token_id,
                          device_map = " auto "
                          )
llm = HuggingFacePipeline(pipeline = model_pipeline)

In [16]:
prompt = hub.pull("rlm/rag-prompt")
def format_docs(docs):
  join_doc = "\n\n".join(doc.page_content for doc in docs )
  return join_doc

rag_chain = (
{"context": retriever | format_docs, "question": RunnablePassthrough()}
| prompt
| llm
| StrOutputParser ()
)

USER_QUESTION = "trận đấu giữa Pháp và Bỉ ở Euro 2024, đội nào thắng, thắng bao nhiêu bàn, ai ghi bàn thắng?"
output = rag_chain.invoke(USER_QUESTION)
answer = output.split('Answer :')[0].strip ()
print(answer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: trận đấu giữa Pháp và Bỉ ở Euro 2024, đội nào thắng, thắng bao nhiêu bàn, ai ghi bàn thắng? 
Context: Pháp đã thắng 2 lần gặp Bỉ gần nhất, tỉ số là 3 -2 ở bán kết UEFA Nations League 2021 
và 1-0 ở bán kết World Cup 2018. Ảnh: UEFA  
Cả Pháp và Bỉ đều chưa đạt phong độ cao nhất tại EURO 2024. Rất khó để dự đoán 
màn thể hiện của hai đội ở  trận này.  
Mặc dù phòng ngự chắc chắn, nhưng sự kém duyên trước khung thành đã làm Les 
Bleus phải chật vật kể từ trận ra quân với Áo.

Lần thứ hai tại EURO 2024 đội tuyển  Pháp  giành chiến thắng nhờ bàn phản lưới 
nhà của đối phương, lần này là tuyển Bỉ tại vòng 1/8 EURO 2024 vào rạng sáng 
1.7. 
HẾT GIỜ!!! Tuyển Pháp giành chiến thắng 1 -0 trước Bỉ, do pha phản lưới nhà của 
Vertonghen. Tuyển Bỉ 